<a href="https://colab.research.google.com/github/zihadbappy/bnfake/blob/main/BanFakeNews_LSTM_BiLSTM_CNN%2BLSTM_CNN%2BBiLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/zihadbappy/FakeDatasets.git
import pandas as pd
Dataset= pd.read_csv('/content/FakeDatasets/banfakenews_no_duplicate.csv')

Cloning into 'FakeDatasets'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 20 (delta 8), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (20/20), done.


In [2]:
df = pd.DataFrame(Dataset)

In [3]:
df = pd.DataFrame(Dataset)


In [4]:
df=df.dropna()

In [5]:
df.groupby('label').agg('count')

,headline,content
label,,
0,1176,1176
1,1175,1175


In [6]:
X=df.content
Y=df.label

In [7]:
Y.value_counts()

0    1176
1    1175
Name: label, dtype: int64

In [8]:
X.shape

(2351,)

In [9]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout
from keras.callbacks import EarlyStopping

Onehot Representation


In [10]:
voc_size=86000

In [11]:
onehot_repr=[one_hot(words,voc_size)for words in X]

In [12]:
len(onehot_repr)

2351

In [13]:
news_length=300
embedded_news=pad_sequences(onehot_repr,padding='pre',maxlen=news_length)
print(embedded_news)

[[    0     0     0 ... 68370 29650 60106]
 [10426 24921 21610 ...  6657 25304 13860]
 [ 4986 79439 51347 ... 52597 53322 55600]
 ...
 [ 9104 80498 53792 ... 48066 43265 32421]
 [37221 77382 12334 ... 61454 38907 14059]
 [    0     0     0 ... 69479 85538 13860]]


In [14]:
embedded_news[0]


array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,

In [15]:
len(embedded_news),Y.shape

(2351, (2351,))

In [16]:
import numpy as np
X_final=np.array(embedded_news)
y_final=np.array(Y)

In [17]:
X_final.shape,y_final.shape

((2351, 300), (2351,))

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.25, random_state=41, stratify=df['label'])

# **LSTM MODEL**

In [19]:
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=news_length))
model1.add(LSTM(100))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=12,batch_size=64,
    callbacks=[EarlyStopping(monitor="val_accuracy",
    min_delta=0,
    patience=3,
    verbose=1,
    mode="auto",
    restore_best_weights=True,)])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 40)           3440000   
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 3,496,501
Trainable params: 3,496,501
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/12
28/28 [==============================] - 14s 418ms/step - loss: 0.6669 - accuracy: 0.6052 - val_loss: 0.5180 - val_accuracy: 0.8741
Epoch 2/12
28/28 [==============================] - 11s 382ms/step - loss: 0.4866 - accuracy: 0.9053 - val_loss: 0.4727 - val_accuracy: 0.8639
Epoch 3/12
28/28 [=

# **CNN+LSTM MODEL**

In [20]:
from keras.layers import Dense,Conv1D,MaxPooling1D

In [21]:
embedding_vecor_length = 40
model2 = Sequential()
model2.add(Embedding(voc_size, embedding_vecor_length, input_length=news_length))
model2.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model2.add(MaxPooling1D(pool_size=2))
model2.add(LSTM(100))
model2.add(Dense(1, activation='sigmoid'))
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model2.summary())
model2.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=12,batch_size=64,
    callbacks=[EarlyStopping(monitor="val_accuracy",
    min_delta=0,
    patience=3,
    verbose=1,
    mode="auto",
    restore_best_weights=True,)])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 300, 40)           3440000   
                                                                 
 conv1d (Conv1D)             (None, 300, 32)           3872      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 150, 32)          0         
 )                                                               
                                                                 
 lstm_1 (LSTM)               (None, 100)               53200     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 3,497,173
Trainable params: 3,497,173
Non-trainable params: 0
____________________________________________

LSTM and CNN+LSTM evaluation

In [22]:
y_pred1=model1.predict(X_test)
y_pred2=model2.predict(X_test)

In [23]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [24]:
print("LSTM accuracy:%.4f"%accuracy_score(y_test,np.around(y_pred1)))
print("CNN+LSTM accuracy:%.4f"%accuracy_score(y_test,np.around(y_pred2)))

LSTM accuracy:0.8963
CNN+LSTM accuracy:0.9286


In [25]:
print("LSTM:\n", classification_report(y_test,np.around(y_pred1),digits=4))
print("\nCNN+LSTM:\n", classification_report(y_test,np.around(y_pred2),digits=4))

LSTM:
               precision    recall  f1-score   support

           0     0.9396    0.8469    0.8909       294
           1     0.8607    0.9456    0.9011       294

    accuracy                         0.8963       588
   macro avg     0.9002    0.8963    0.8960       588
weighted avg     0.9002    0.8963    0.8960       588


CNN+LSTM:
               precision    recall  f1-score   support

           0     0.9532    0.9014    0.9266       294
           1     0.9065    0.9558    0.9305       294

    accuracy                         0.9286       588
   macro avg     0.9298    0.9286    0.9285       588
weighted avg     0.9298    0.9286    0.9285       588



In [26]:
print("LSTM confusion matrix:\n ",confusion_matrix(y_test,np.around(y_pred1)))
print("\nCNN+LSTM confusion matrix:\n",confusion_matrix(y_test,np.around(y_pred2)))

LSTM confusion matrix:
  [[249  45]
 [ 16 278]]

CNN+LSTM confusion matrix:
 [[265  29]
 [ 13 281]]


#**Bi-LSTM MODEL**

In [27]:
embedding_vector_features=40
model3=Sequential()
model3.add(Embedding(voc_size,embedding_vector_features,input_length=news_length))
model3.add(Bidirectional(LSTM(100)))
model3.add(Dropout(0.3))
model3.add(Dense(1,activation='sigmoid'))
model3.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model3.summary())
model3.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=12,batch_size=64,
    callbacks=[EarlyStopping(monitor="val_accuracy",
    min_delta=0,
    patience=3,
    verbose=1,
    mode="auto",
    restore_best_weights=True,)])

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 300, 40)           3440000   
                                                                 
 bidirectional (Bidirectiona  (None, 200)              112800    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 200)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 201       
                                                                 
Total params: 3,553,001
Trainable params: 3,553,001
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/12
28/28 [==============================] - 23s 666ms/step - loss: 0.6892 - accuracy: 0.5740 - va

## **CNN+BiLSTM MODEL**

In [28]:
embedding_vecor_length = 40
model4 = Sequential()
model4.add(Embedding(voc_size, embedding_vecor_length, input_length=news_length))
model4.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model4.add(MaxPooling1D(pool_size=2))
model4.add(Bidirectional(LSTM(100)))
model4.add(Dense(1, activation='sigmoid'))
model4.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model4.summary())
model4.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=12,batch_size=64,
    callbacks=[EarlyStopping(monitor="val_accuracy",
    min_delta=0,
    patience=3,
    verbose=1,
    mode="auto",
    restore_best_weights=True,)])

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 300, 40)           3440000   
                                                                 
 conv1d_1 (Conv1D)           (None, 300, 32)           3872      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 150, 32)          0         
 1D)                                                             
                                                                 
 bidirectional_1 (Bidirectio  (None, 200)              106400    
 nal)                                                            
                                                                 
 dense_3 (Dense)             (None, 1)                 201       
                                                                 
Total params: 3,550,473
Trainable params: 3,550,473
No

In [29]:
y_pred3=model3.predict(X_test)
y_pred4=model4.predict(X_test)

In [30]:
print("BiLSTM accuracy:%.4f"%accuracy_score(y_test,np.around(y_pred3)))
print("CNN+BiLSTM accuracy:%.4f"%accuracy_score(y_test,np.around(y_pred4)))

BiLSTM accuracy:0.9269
CNN+BiLSTM accuracy:0.9184


In [31]:
print("BiLSTM:\n",classification_report(y_test,np.around(y_pred3),digits=4))
print("\nCNN+BiLSTM:\n",classification_report(y_test,np.around(y_pred4),digits=4))

BiLSTM:
               precision    recall  f1-score   support

           0     0.9498    0.9014    0.9250       294
           1     0.9061    0.9524    0.9287       294

    accuracy                         0.9269       588
   macro avg     0.9280    0.9269    0.9268       588
weighted avg     0.9280    0.9269    0.9268       588


CNN+BiLSTM:
               precision    recall  f1-score   support

           0     0.9020    0.9388    0.9200       294
           1     0.9362    0.8980    0.9167       294

    accuracy                         0.9184       588
   macro avg     0.9191    0.9184    0.9183       588
weighted avg     0.9191    0.9184    0.9183       588



In [32]:
print("BiLSTM confusion matrix:\n",confusion_matrix(y_test,np.around(y_pred3)))
print("\nCNN+BiLSTM confusion matrix:\n",confusion_matrix(y_test,np.around(y_pred4)))

BiLSTM confusion matrix:
 [[265  29]
 [ 14 280]]

CNN+BiLSTM confusion matrix:
 [[276  18]
 [ 30 264]]
